In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://mongo:27017/')
db = client.vk
collection = db['toxic_images']

In [ ]:
from kafka import KafkaConsumer
from json import loads
import time

consumer = KafkaConsumer(
     'images',
     bootstrap_servers=['kafka:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     auto_commit_interval_ms=1000)

In [ ]:
from Analyzer import Analyzer
import os

vectorizer_path = '/home/jovyan/big-data-prj/ml_text_processing/weights/vectorizer.pkl'
regressor_path = '/home/jovyan/big-data-prj/ml_text_processing/weights/regressor.pkl'

analyzer = Analyzer(vectorizer_path, regressor_path)

base_path = '/home/jovyan/images'

for message in consumer:
    try:
        img_name = message.value.decode("utf-8")
    except Exception as e:
        print(e)
        continue
    full_path = os.path.join(base_path, img_name)

    
    text, toxicity = analyzer.predict_image_toxicity(full_path, None)
    
    entry = {'text': text.strip(), 'img_name': img_name, 'toxicity': toxicity}
    _id = collection.insert_one(entry).inserted_id
    print(f"{_id}: {img_name}, {text} - {toxicity}")